In [1]:
import os
import copy
import tempfile
import json
from subprocess import call
from diConstants import (HG19_ALL_CHROMS, MM9_ALL_CHROMS,
    HG19_TRAIN_CHROMS, MM9_TRAIN_CHROMS,
    VALID_CHROMS, TEST_CHROMS) 

import models
import modelTemplates

Using Theano backend.


In [2]:
GM_MARKS = ['H3K27AC'] # Is this output mark?
#GM_MARKS = ['H3K27AC', 'H3K4ME1', 'H3K4ME3', 'H3K27ME3', 'H3K36ME3']
modelList = []
def init_test_GM18526():
    for test_cell_line in ['GM18526']:
        for subsample_target_string in ['0.5e6']:
            for predict_binary_output in [True, False]: # original : TRUE , FALSE   
                for output_mark in GM_MARKS:                            

                    model_params = modelTemplates.make_model_params(
                                #### Prediction model(keras ver) specification ####
                                model_library='keras',
                                model_class='SeqToPoint',
                                model_type='dcgan',
                                model_specific_params={
                                    'num_filters': 6,
                                    'filter_length': 51
                                },
                                compile_params={            
                                    'optimizer': 'adagrad'
                                },
            
                                #### Dataset specification ####
                                dataset_params={
                                    'train_dataset_name': 'GM12878_5+1marks-K4me3_all',
                                    'test_dataset_name': '%s_5+1marks-K4me3_all' % test_cell_line, #(1)
                                    'num_train_examples': 1000,
                                    'seq_length': 1001,
                                    'peak_fraction': 0.5,                            
                                    'train_X_subsample_target_string': subsample_target_string, #(2)
                                    'num_bins_to_test': None,
                                    'train_chroms': HG19_ALL_CHROMS, # GM-12878 ch1 ~ ch22
                                    'test_chroms': HG19_ALL_CHROMS, # CM-18526 ch1 ~ ch22
                                    'only_chr1': True
                                },
                                output_marks=[output_mark],
                                train_params={
                                    'nb_epoch': 1,
                                    'batch_size': 100
                                },
                                predict_binary_output=predict_binary_output,
                                zero_out_non_bins=True,
                                generate_bigWig=True)

                    init_model(model_params)

def init_model(model_params):
    m = models.SeqModel.instantiate_model(model_params)
    modelList.append([m,model_params])
    print '\n\n\n'

In [3]:
init_test_GM18526()

Model output mark: ['H3K27AC']
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, None, 6)       1842        convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, 6)       0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, None, 1)       6007        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, None, 1)       0           convolution1d_2[0][0]            
Total params: 7,849
Trainable params: 7,849
Non-trainable pa

In [4]:
def train_model():
    print 'Number of Model:', len(modelList)
    for model, params in modelList:
        print model
        model.compile_and_train_model()
    
train_model()

Number of Model: 2
mid: 500
mid: 500
mid: 500
model.fit() start.....
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 5s - loss: 0.9072 - val_loss: 0.3488
...Done!
mid: 500
mid: 500
mid: 500
model.fit() start.....
Train on 800 samples, validate on 200 samples
Epoch 1/1
800/800 [==============================] - 5s - loss: 3.7677 - val_loss: 1.1213
...Done!


In [ ]:
def evaluate_model():
    for model, params in modelList:
        print model.evaluate_model()

#evaluate_model()

mid: 500
mid: 500
mid: 500
train samples - Original peaks vs. true peaks:
mid: 500
AUC 0.786; AUPRC 0.839
train samples - Predicted peaks vs. true peaks:
AUC 0.973; AUPRC 0.974
Evaluate model on testing data
mid: 500
mid: 500
mid: 500
test samples - Original peaks vs. true peaks:
mid: 500
AUC 0.662; AUPRC 0.711
test samples - Predicted peaks vs. true peaks:
AUC 0.967; AUPRC 0.966
Test model on genome index 0 ...
Test chr1, 9.97E+06 bins - Original:
AUC 0.600; AUPRC 0.279
Test chr1, 9.97E+06 bins - Denoised, all signal:
AUC 0.963; AUPRC 0.771
final results {u'orig_all': {u'chr1': {'precision_curves': [[0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6, 0.7, 0.7999999999999999, 0.8999999999999999, 0.9999999999999999]], 'AUC': [0.6000754360972312], 'Y_pos_frac': [0.06241609849685417], 'AUPRC': [0.27858695550065293], 'recall_curves': [[0.27690144466406336, 0.27690144466406336, 0.27690144466406336, 0.27690144466406336, 0.27690144466406336, 0.27690144466406336, 0.17698661395812243, 0.097649006090